# Exploration of Resident Positions

In [1]:
import plotly.express as px

import DataRetriever as dr

retriever = dr.DataRetriever()

year_2 = retriever.get_data('All-Subsystems-minute-Year2.pkl')
metadata = retriever.get_data('metadata-year2.pkl')
metadata.rename(columns={'Unnamed: 0': 'Attribute'}, inplace=True)

## Find relevant attributes

In [2]:
parent_a_positions = metadata[metadata["Description"].str.contains(pat='parent A is simulated as being')]
parent_b_positions = metadata[metadata["Description"].str.contains(pat='parent B is simulated as being')]
child_a_positions = metadata[metadata["Description"].str.contains(pat='child A is simulated as being')]
child_b_positions = metadata[metadata["Description"].str.contains(pat='child B is simulated as being')]

# Get dict of DataFrames - each one containing information about a different resident
positions_dict = {'ParentA': parent_a_positions['Attribute'].tolist(),
                  'ParentB': parent_b_positions['Attribute'].tolist(),
                  'ChildA': child_a_positions['Attribute'].tolist(),
                  'ChildB': child_b_positions['Attribute'].tolist()}

In [3]:
# Get a list of relevant attributes
positions = list()
for key, value in positions_dict.items():
    for subvalue in positions_dict[key]:
        positions.append(subvalue)

In [ ]:
# Use the identified attributes from the Metadata DataFrame to get the actual data from the Year2 DataFrame
position_data = year_2[positions]
position_data.index = position_data.index.floor('min')
position_data["Date"] = position_data.index.date
position_data["Time"] = position_data.index.time
position_data["Weekend"] = position_data.index.weekday > 5

In [ ]:
# Rename each attribute to more sensible names
position_data.rename(columns={'Load_StatusSensHeatPrntAUP': 'ParentAUp',
                              'Load_StatusSensHeatPrntADOWN': 'ParentADown',
                              'Load_StatusSensHeatPrntBUP': 'ParentBUp',
                              'Load_StatusSensHeatPrntBDOWN': 'ParentBDown',
                              'Load_StatusSensHeatChildAUP': 'ChildAUp',
                              'Load_StatusSensHeatChildADOWN': 'ChildADown',
                              'Load_StatusSensHeatChildBUP': 'ChildBUp',
                              'Load_StatusSensHeatChildBDOWN': 'ChildBDown'}, inplace=True)

In [ ]:
# Replace values for each resident being on the second floor in order to easily
position_data["ParentAUp"].replace(to_replace=1.0, value=2.0, inplace=True)
position_data["ParentBUp"].replace(to_replace=1.0, value=2.0, inplace=True)
position_data["ChildAUp"].replace(to_replace=1.0, value=2.0, inplace=True)
position_data["ChildBUp"].replace(to_replace=1.0, value=2.0, inplace=True)

# Create a single attribute containing the value for where each inhabitant is
position_data["ParentAPosition"] = position_data["ParentAUp"] + position_data["ParentADown"]
position_data["ParentBPosition"] = position_data["ParentBUp"] + position_data["ParentBDown"]
position_data["ChildAPosition"] = position_data["ChildAUp"] + position_data["ChildADown"]
position_data["ChildBPosition"] = position_data["ChildBUp"] + position_data["ChildBDown"]

# Replace the number values with explanatory text
position_data.replace({'ParentAPosition': {0: 'Not Home', 1: 'Downstairs', 2: 'Upstairs'}}, inplace=True)
position_data.replace({'ParentBPosition': {0: 'Not Home', 1: 'Downstairs', 2: 'Upstairs'}}, inplace=True)
position_data.replace({'ChildAPosition': {0: 'Not Home', 1: 'Downstairs', 2: 'Upstairs'}}, inplace=True)
position_data.replace({'ChildBPosition': {0: 'Not Home', 1: 'Downstairs', 2: 'Upstairs'}}, inplace=True)
position_data

In [ ]:
fig = px.scatter(data_frame=position_data,
                 x='Time',
                 y='ParentAPosition',
                 color='Weekend',
                 opacity=0.025)
fig.update_layout(title='Position of Parent A by time during weekdays')
fig.update_xaxes(type='category')
fig.update_yaxes(type='category')
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scattergl(x=position_data.index[:10080], y=position_data["ParentAPosition"], name="ParentA"))
fig.add_trace(go.Scattergl(x=position_data.index[:10080], y=position_data["ParentBPosition"], name="ParentB"))
fig.add_trace(go.Scattergl(x=position_data.index[:10080], y=position_data["ParentBPosition"], name="ChildA"))
fig.add_trace(go.Scattergl(x=position_data.index[:10080], y=position_data["ParentBPosition"], name="ChildB"))
fig.show()